# 📰 Sistema de Recuperación de Noticias RPP
## News Retrieval System + LangChain + ChromaDB

**Objetivo:** Construir un sistema end-to-end de recuperación de noticias desde RSS de RPP Perú, utilizando embeddings, ChromaDB y LangChain.

### Pipeline:
```
RSS Feed → Tokenización → Embeddings → ChromaDB → Retrieval → Resultados
```

---

## 📦 PASO 0: Instalación de Librerías

In [1]:
# Instalación silenciosa de todas las librerías necesarias
!pip install feedparser tiktoken sentence-transformers chromadb langchain langchain-community pandas numpy -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 119.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 112.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Imports necesarios
import feedparser
import tiktoken
import pandas as pd
import numpy as np
import json
import os
from datetime import datetime
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document

print("✅ Todas las librerías importadas correctamente")

✅ Todas las librerías importadas correctamente


In [3]:
#!pip install google.colab

In [4]:
# Montar Google Drive
from google.colab import drive
import os

drive.mount('/content/drive', force_remount=True)
print("✅ Google Drive montado exitosamente")

Mounted at /content/drive
✅ Google Drive montado exitosamente


In [5]:
BASE_PATH = "/content/drive/MyDrive/news-query_RPP-lab"

# Crear estructura de carpetas en Drive
os.makedirs(BASE_PATH, exist_ok=True)
os.makedirs(f"{BASE_PATH}/data", exist_ok=True)
os.makedirs(f"{BASE_PATH}/output", exist_ok=True)
os.makedirs(f"{BASE_PATH}/chroma_db", exist_ok=True)

print("📁 Estructura de carpetas creada en Google Drive:")
print(f"   📍 Ubicación: {BASE_PATH}")
print(f"   - {BASE_PATH}/data/       → Datos crudos y procesados")
print(f"   - {BASE_PATH}/output/     → Resultados de queries")
print(f"   - {BASE_PATH}/chroma_db/  → Base de datos vectorial")


📁 Estructura de carpetas creada en Google Drive:
   📍 Ubicación: /content/drive/MyDrive/news-query_RPP-lab
   - /content/drive/MyDrive/news-query_RPP-lab/data/       → Datos crudos y procesados
   - /content/drive/MyDrive/news-query_RPP-lab/output/     → Resultados de queries
   - /content/drive/MyDrive/news-query_RPP-lab/chroma_db/  → Base de datos vectorial


---
## 📡 PASO 1: Carga de Datos desde RSS Feed

Extraemos las **50 noticias más recientes** del feed RSS de RPP Perú.

In [6]:
# Drive ya montado en celda anterior
print("✅ Usando Drive montado")


✅ Usando Drive montado


In [7]:
def load_rss_feed(url, max_items=50):
    # Parsear el feed RSS
    feed = feedparser.parse(url)

    news_items = []

    # Extraer información de cada noticia
    for entry in feed.entries[:max_items]:
        news_item = {
            'title': entry.get('title', 'Sin título'),
            'description': entry.get('summary', 'Sin descripción'),
            'link': entry.get('link', ''),
            'published': entry.get('published', 'Fecha no disponible')
        }
        news_items.append(news_item)

    print(f"✅ {len(news_items)} noticias cargadas exitosamente")
    return news_items

In [8]:
# Cargar noticias del RSS de RPP
RSS_URL = "https://rpp.pe/rss"
news_items = load_rss_feed(RSS_URL, max_items=50)

# Mostrar primeras 3 noticias como muestra
print("\n📰 Primeras 3 noticias:")
print("=" * 100)
for i, item in enumerate(news_items[:3], 1):
    print(f"\n{i}. {item['title']}")
    print(f"   📅 {item['published']}")
    print(f"   📝 {item['description'][:100]}...")
    print(f"   🔗 {item['link']}")

✅ 50 noticias cargadas exitosamente

📰 Primeras 3 noticias:

1. Cúal fue el último temblor en México hoy 14 de noviembre según SSN
   📅 Wed, 12 Nov 2025 06:21:35 -0500
   📝 Cuál es el ultimo temblor en México y CDMX registrado segun el Servicio Sismológico Nacional (SSN) h...
   🔗 https://rpp.pe/mundo/mexico/cual-fue-el-ultimo-temblor-en-mexico-hoy-12-de-noviembre-segun-ssn-live-3040

2. Vania Torres tras consagrarse campeona mundial de surf: “Siento que he entrenado casi toda mi vida para esto”
   📅 Fri, 14 Nov 2025 13:36:22 -0500
   📝 En el programa Encendidos, la surfista peruana reconoció que asistió a la competencia con la clara i...
   🔗 https://rpp.pe/multideportes/mas-deportes/vania-torres-tras-consagrarse-campeona-mundial-de-sur-siento-ya-que-he-entrenado-casi-toda-mi-vida-para-esto-noticia-1663823

3. ¿Dónde votar en las elecciones en Chile? Consulta tu local y mesa en Servel con tu RUT
   📅 Fri, 14 Nov 2025 13:00:52 -0500
   📝 Consulta en Servel (Servicio Electoral de Chile)

In [9]:
# Convertir a DataFrame para mejor visualización
df_news = pd.DataFrame(news_items)

print("\n📊 DataFrame de Noticias:")
print(df_news.head())
print(f"\nDimensiones: {df_news.shape[0]} filas x {df_news.shape[1]} columnas")


📊 DataFrame de Noticias:
                                               title  \
0  Cúal fue el último temblor en México hoy 14 de...   
1  Vania Torres tras consagrarse campeona mundial...   
2  ¿Dónde votar en las elecciones en Chile? Consu...   
3  Gutiérrez sobre cárcel para manifestantes con ...   
4  ¡Ya está en semifinales! Ignacio Buse venció a...   

                                         description  \
0  Cuál es el ultimo temblor en México y CDMX reg...   
1  En el programa Encendidos, la surfista peruana...   
2  Consulta en Servel (Servicio Electoral de Chil...   
3  Josué Gutiérrez, defensor del Pueblo, consider...   
4  Ignacio Buse se impuso 2-0 al brasileño Gonzal...   

                                                link  \
0  https://rpp.pe/mundo/mexico/cual-fue-el-ultimo...   
1  https://rpp.pe/multideportes/mas-deportes/vani...   
2  https://rpp.pe/mundo/chile/chile-servel-donde-...   
3  https://rpp.pe/politica/judiciales/gutierrez-s...   
4  https://rpp.pe/te

In [10]:
# Guardar datos en Drive
with open(f"{BASE_PATH}/data/rss_raw.json", 'w', encoding='utf-8') as f:
    json.dump(news_items, f, ensure_ascii=False, indent=2)

df_news.to_csv(f"{BASE_PATH}/data/rss_raw.csv", index=False, encoding='utf-8')

print("\n💾 Datos guardados en Google Drive:")
print(f"   📄 {BASE_PATH}/data/rss_raw.json")
print(f"   📄 {BASE_PATH}/data/rss_raw.csv")



💾 Datos guardados en Google Drive:
   📄 /content/drive/MyDrive/news-query_RPP-lab/data/rss_raw.json
   📄 /content/drive/MyDrive/news-query_RPP-lab/data/rss_raw.csv


---
## 🔢 PASO 2: Tokenización con tiktoken

Analizamos el conteo de tokens para determinar si se necesita chunking.

In [11]:
def count_tokens(text, encoding_name="cl100k_base"):
    """
    Cuenta el número de tokens en un texto usando tiktoken.

    Args:
        text: Texto a tokenizar
        encoding_name: Nombre del encoding (cl100k_base para GPT-4)

    Returns:
        Número de tokens
    """
    encoding = tiktoken.get_encoding(encoding_name)
    tokens = encoding.encode(text)
    return len(tokens)

def format_news_for_embedding(news_item):
    """
    Formatea una noticia combinando título y descripción para embedding.

    Args:
        news_item: Diccionario con información de la noticia

    Returns:
        String formateado: "título. descripción"
    """
    return f"{news_item['title']}. {news_item['description']}"

print("✅ Funciones de tokenización definidas")

✅ Funciones de tokenización definidas


In [12]:
# Analizar tokens en un artículo de muestra
sample_article = format_news_for_embedding(news_items[0])

print("📝 Artículo de muestra:")
print(sample_article)
print("\n" + "="*100)

# Contar tokens
sample_tokens = count_tokens(sample_article)
print(f"\n🔢 Número de tokens: {sample_tokens}")
print(f"¿Necesita chunking? (>8192): {'❌ NO' if sample_tokens < 8192 else '✅ SÍ'}")

📝 Artículo de muestra:
Cúal fue el último temblor en México hoy 14 de noviembre según SSN. Cuál es el ultimo temblor en México y CDMX registrado segun el Servicio Sismológico Nacional (SSN) hoy 14 de noviembre del 2025. Consulta los últimos sismos EN VIVO para México aquí.


🔢 Número de tokens: 75
¿Necesita chunking? (>8192): ❌ NO


In [13]:
# Análisis estadístico de tokens en todas las noticias
token_counts = [count_tokens(format_news_for_embedding(item)) for item in news_items]

print("📊 Estadísticas de Tokens:")
print("=" * 50)
print(f"   Promedio:  {np.mean(token_counts):.2f} tokens")
print(f"   Mínimo:    {np.min(token_counts)} tokens")
print(f"   Máximo:    {np.max(token_counts)} tokens")
print(f"   Mediana:   {np.median(token_counts):.2f} tokens")
print(f"   Desv. Est: {np.std(token_counts):.2f} tokens")
print("\n💡 Conclusión: Los artículos son cortos, NO se necesita chunking")

# Guardar análisis en Drive
token_analysis = {
    'promedio': float(np.mean(token_counts)),
    'minimo': int(np.min(token_counts)),
    'maximo': int(np.max(token_counts)),
    'mediana': float(np.median(token_counts))
}

with open(f"{BASE_PATH}/output/token_analysis.json", 'w', encoding='utf-8') as f:
    json.dump(token_analysis, f, ensure_ascii=False, indent=2)

print(f"\n💾 Análisis guardado en: {BASE_PATH}/output/token_analysis.json")
print("\n💾 Análisis guardado en: output/token_analysis.json")

📊 Estadísticas de Tokens:
   Promedio:  79.86 tokens
   Mínimo:    44 tokens
   Máximo:    119 tokens
   Mediana:   75.00 tokens
   Desv. Est: 18.52 tokens

💡 Conclusión: Los artículos son cortos, NO se necesita chunking

💾 Análisis guardado en: /content/drive/MyDrive/news-query_RPP-lab/output/token_analysis.json

💾 Análisis guardado en: output/token_analysis.json


---
## 🤖 PASO 3: Generación de Embeddings

Usamos el modelo **sentence-transformers/all-MiniLM-L6-v2** para generar embeddings de 384 dimensiones.

In [14]:
# Inicializar modelo de embeddings
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
print(f"🤖 Cargando modelo: {MODEL_NAME}")
embedding_model = SentenceTransformer(MODEL_NAME)
print(f"   Dimensión de embeddings: {embedding_model.get_sentence_embedding_dimension()}")

🤖 Cargando modelo: sentence-transformers/all-MiniLM-L6-v2


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

   Dimensión de embeddings: 384


In [15]:
# Preparar textos para embedding (título + descripción)
texts_for_embedding = [format_news_for_embedding(item) for item in news_items]

print(f"📝 Textos preparados: {len(texts_for_embedding)} artículos")
print(f"\nEjemplo de texto formateado:")
print(texts_for_embedding[0][:200] + "...")

📝 Textos preparados: 50 artículos

Ejemplo de texto formateado:
Cúal fue el último temblor en México hoy 14 de noviembre según SSN. Cuál es el ultimo temblor en México y CDMX registrado segun el Servicio Sismológico Nacional (SSN) hoy 14 de noviembre del 2025. Con...


In [16]:
# Generar embeddings para todos los textos
print("🔄 Generando embeddings...")

embeddings = embedding_model.encode(
    texts_for_embedding,
    show_progress_bar=True,
    convert_to_numpy=True
)

print(f"\n✅ Embeddings generados: {embeddings.shape}")

# Guardar embeddings en Drive
np.save(f"{BASE_PATH}/data/embeddings.npy", embeddings)
print(f"💾 Embeddings guardados en: {BASE_PATH}/data/embeddings.npy")


🔄 Generando embeddings...


Batches:   0%|          | 0/2 [00:00<?, ?it/s]


✅ Embeddings generados: (50, 384)
💾 Embeddings guardados en: /content/drive/MyDrive/news-query_RPP-lab/data/embeddings.npy


In [17]:
# Guardar datos procesados en Drive
processed_data = {
    'news_items': news_items,
    'texts': texts_for_embedding,
    'embeddings_shape': embeddings.shape,
    'model_name': MODEL_NAME
}

with open(f"{BASE_PATH}/data/processed_news.json", 'w', encoding='utf-8') as f:
    json.dump(processed_data, f, ensure_ascii=False, indent=2)

print(f"💾 Datos procesados guardados en: {BASE_PATH}/data/processed_news.json")


💾 Datos procesados guardados en: /content/drive/MyDrive/news-query_RPP-lab/data/processed_news.json


---
## 💾 PASO 4: Almacenamiento en ChromaDB

Creamos una colección persistente en ChromaDB para almacenar documentos, metadatos y embeddings.

In [18]:
# Inicializar ChromaDB en Drive
CHROMA_PATH = f"{BASE_PATH}/chroma_db"

print(f"💾 Inicializando ChromaDB en Drive...")
print(f"   📍 Ubicación: {CHROMA_PATH}")

chroma_client = chromadb.PersistentClient(path=CHROMA_PATH)

print("✅ ChromaDB inicializado")


💾 Inicializando ChromaDB en Drive...
   📍 Ubicación: /content/drive/MyDrive/news-query_RPP-lab/chroma_db
✅ ChromaDB inicializado


In [19]:
# Crear o obtener colección (si existe, la reinicia)
COLLECTION_NAME = "rpp_news"

# Eliminar colección anterior si existe
try:
    chroma_client.delete_collection(name=COLLECTION_NAME)
    print(f"🗑️  Colección anterior '{COLLECTION_NAME}' eliminada")
except:
    pass

# Crear nueva colección
collection = chroma_client.create_collection(
    name=COLLECTION_NAME,
    metadata={"description": "Noticias de RPP Perú"}
)

print(f"✅ Colección '{COLLECTION_NAME}' creada exitosamente")

🗑️  Colección anterior 'rpp_news' eliminada
✅ Colección 'rpp_news' creada exitosamente


In [20]:
# Preparar metadatos para ChromaDB
metadatas = [
    {
        'title': item['title'],
        'description': item['description'],
        'link': item['link'],
        'published': item['published']
    }
    for item in news_items
]

# Generar IDs únicos
ids = [f"news_{i}" for i in range(len(news_items))]

print(f"📝 Metadatos preparados: {len(metadatas)} entradas")
print(f"   IDs generados: {ids[:5]} ... {ids[-1]}")

📝 Metadatos preparados: 50 entradas
   IDs generados: ['news_0', 'news_1', 'news_2', 'news_3', 'news_4'] ... news_49


In [21]:
# Agregar documentos a la colección (upsert)
print("⬆️  Insertando documentos en ChromaDB...")

collection.add(
    documents=texts_for_embedding,
    embeddings=embeddings.tolist(),
    metadatas=metadatas,
    ids=ids
)

print(f"✅ {len(ids)} documentos insertados exitosamente")
print(f"\n📊 Información de la colección:")
print(f"   Nombre: {collection.name}")
print(f"   Total documentos: {collection.count()}")

⬆️  Insertando documentos en ChromaDB...
✅ 50 documentos insertados exitosamente

📊 Información de la colección:
   Nombre: rpp_news
   Total documentos: 50


---
## 🔍 PASO 5: Sistema de Búsqueda (Query)

Implementamos búsquedas por similitud semántica y mostramos resultados en DataFrames.

In [22]:
def search_news(query_text, n_results=10):
    """
    Realiza búsqueda por similitud semántica en la colección de noticias.

    Args:
        query_text: Texto de consulta
        n_results: Número de resultados a retornar

    Returns:
        DataFrame con las noticias más relevantes
    """
    print(f"🔍 Buscando: '{query_text}'")
    print(f"   Top {n_results} resultados\n")

    # Generar embedding del query
    query_embedding = embedding_model.encode([query_text])[0]

    # Realizar búsqueda en ChromaDB
    results = collection.query(
        query_embeddings=[query_embedding.tolist()],
        n_results=n_results
    )

    # Extraer metadatos
    metadatas = results['metadatas'][0]
    distances = results['distances'][0]

    # Crear DataFrame
    df_results = pd.DataFrame([
        {
            'title': meta['title'],
            'description': meta['description'],
            'link': meta['link'],
            'date_published': meta['published'],
            'similarity_score': 1 - dist  # Convertir distancia a similitud
        }
        for meta, dist in zip(metadatas, distances)
    ])

    print(f"✅ {len(df_results)} resultados encontrados\n")

    return df_results

print("✅ Función de búsqueda definida")

✅ Función de búsqueda definida


### 🎯 Query 1: Noticias de Economía

In [24]:
# Query 1: Economía
query_1 = "Últimas noticias de economía"
results_economia = search_news(query_1, n_results=10)

print("="*100)
print(results_economia[['title', 'date_published', 'similarity_score']].to_string(index=False))
print("="*100)

# Guardar resultados en Drive
results_economia.to_csv(f"{BASE_PATH}/output/query_results_economia.csv", index=False, encoding='utf-8')
print(f"\n💾 Resultados guardados en: {BASE_PATH}/output/query_results_economia.csv")


🔍 Buscando: 'Últimas noticias de economía'
   Top 10 resultados

✅ 10 resultados encontrados

                                                                                                                      title                  date_published  similarity_score
                                      BCRP mantiene tasa de interés de referencia en 4.25 % para noviembre: ¿A qué se debe? Fri, 14 Nov 2025 12:28:48 -0500         -0.192940
                                                                     Trabajador murió dentro de obra de colegio en Chiclayo Fri, 14 Nov 2025 11:30:31 -0500         -0.305275
                      ¿Menos pirotécnicos en estas fiestas? CCL advierte contracción de importaciones y explica las razones Fri, 14 Nov 2025 12:15:27 -0500         -0.326764
'Safaris de la muerte': escándalo en Italia por denuncia de supuestos viajes para disparar civiles durante guerra de Bosnia Fri, 14 Nov 2025 13:16:36 -0500         -0.357291
                                    

### 🏛️ Query 2: Noticias de Política

In [25]:
# Query 2: Política
query_2 = "Noticias sobre política y gobierno"
results_politica = search_news(query_2, n_results=10)

print("="*100)
print(results_politica[['title', 'date_published', 'similarity_score']].to_string(index=False))
print("="*100)

# Guardar resultados en Drive
results_politica.to_csv(f"{BASE_PATH}/output/query_results_politica.csv", index=False, encoding='utf-8')
print(f"\n💾 Resultados guardados en: {BASE_PATH}/output/query_results_politica.csv")


🔍 Buscando: 'Noticias sobre política y gobierno'
   Top 10 resultados

✅ 10 resultados encontrados

                                                                                                                              title                  date_published  similarity_score
                Golpe al crimen: caen ‘Los Furiosos del Sur’, banda criminal dedicada a la fabricación y tráfico de billetes falsos Fri, 14 Nov 2025 11:45:13 -0500         -0.117553
Marisol Pérez Tello, precandidata presidencial de Primero la Gente: "Lo primero que se va a hacer es derogar" las leyes 'procrimen' Fri, 14 Nov 2025 12:00:05 -0500         -0.140826
                Renzo Reggiardo informó que hoy regirá la intangibilidad en la plaza San Martín ante convocatoria de movilizaciones Fri, 14 Nov 2025 11:11:51 -0500         -0.159161
                                                  Rusia espera que EE.UU. no tome medidas para desestabilizar Venezuela y el Caribe Fri, 14 Nov 2025 12:56:54 -0500         

### 🎵 Query 3: Noticias de Entretenimiento

In [26]:
# Query 3: Entretenimiento
query_3 = "Noticias de música y conciertos"
results_entretenimiento = search_news(query_3, n_results=10)

print("="*100)
print(results_entretenimiento[['title', 'date_published', 'similarity_score']].to_string(index=False))
print("="*100)

# Guardar resultados en Drive
results_entretenimiento.to_csv(f"{BASE_PATH}/output/query_results_entretenimiento.csv", index=False, encoding='utf-8')
print(f"\n💾 Resultados guardados en: {BASE_PATH}/output/query_results_entretenimiento.csv")


🔍 Buscando: 'Noticias de música y conciertos'
   Top 10 resultados

✅ 10 resultados encontrados

                                                                                                                       title                  date_published  similarity_score
                                   Fans de Shakira duermen en los exteriores del Estadio Nacional antes del primer concierto Fri, 14 Nov 2025 12:16:54 -0500          0.088972
                         Fans de Shakira pasan la noche afuera del Estadio Nacional a la espera del primer concierto en Lima Fri, 14 Nov 2025 13:32:21 -0500          0.036342
                                              Club de fans de Shakira dan detalles de los tres conciertos de Shakira en Lima Fri, 14 Nov 2025 12:16:54 -0500          0.021832
 Susy Díaz emocionada por caminar con Shakira en su concierto en Lima: "Es una bendición de Dios. Para mí, ella es la mejor" Fri, 14 Nov 2025 12:00:58 -0500         -0.054457
                       Shaki

---
## 🔗 PASO 6: Orquestación con LangChain

Implementamos el pipeline completo usando LangChain para mayor modularidad y extensibilidad.

In [27]:
# Inicializar embeddings de LangChain
print("🔗 Inicializando componentes de LangChain...")

langchain_embeddings = HuggingFaceEmbeddings(
    model_name=MODEL_NAME,
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

print("✅ Embeddings de LangChain inicializados")

🔗 Inicializando componentes de LangChain...


/tmp/ipython-input-3201800989.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  langchain_embeddings = HuggingFaceEmbeddings(


✅ Embeddings de LangChain inicializados


In [28]:
# Crear documentos de LangChain
print("📝 Creando documentos de LangChain...")

langchain_documents = [
    Document(
        page_content=text,
        metadata=meta
    )
    for text, meta in zip(texts_for_embedding, metadatas)
]

print(f"✅ {len(langchain_documents)} documentos creados")
print(f"\nEjemplo de documento:")
print(f"   Content: {langchain_documents[0].page_content[:100]}...")
print(f"   Metadata: {langchain_documents[0].metadata}")

📝 Creando documentos de LangChain...
✅ 50 documentos creados

Ejemplo de documento:
   Content: Cúal fue el último temblor en México hoy 14 de noviembre según SSN. Cuál es el ultimo temblor en Méx...
   Metadata: {'title': 'Cúal fue el último temblor en México hoy 14 de noviembre según SSN', 'description': 'Cuál es el ultimo temblor en México y CDMX registrado segun el Servicio Sismológico Nacional (SSN) hoy 14 de noviembre del 2025. Consulta los últimos sismos EN VIVO para México aquí.', 'link': 'https://rpp.pe/mundo/mexico/cual-fue-el-ultimo-temblor-en-mexico-hoy-12-de-noviembre-segun-ssn-live-3040', 'published': 'Wed, 12 Nov 2025 06:21:35 -0500'}


In [29]:
# Crear VectorStore de LangChain en Drive
print("💾 Creando VectorStore de LangChain...")

vectorstore = Chroma.from_documents(
    documents=langchain_documents,
    embedding=langchain_embeddings,
    collection_name="rpp_langchain",
    persist_directory=CHROMA_PATH
)

print("✅ VectorStore creado y persistido en Drive")


💾 Creando VectorStore de LangChain...
✅ VectorStore creado y persistido en Drive


In [30]:
# Crear retriever de LangChain
print("🔍 Configurando retriever de LangChain...")

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 10}
)

print("✅ Retriever configurado (Top 10 resultados)")

🔍 Configurando retriever de LangChain...
✅ Retriever configurado (Top 10 resultados)


In [31]:
# Función de búsqueda con LangChain
def langchain_search(query_text):
    """
    Realiza búsqueda usando el retriever de LangChain.

    Args:
        query_text: Texto de consulta

    Returns:
        DataFrame con resultados
    """
    print(f"🔍 [LangChain] Buscando: '{query_text}'\n")

    # Obtener documentos relevantes
    docs = retriever.get_relevant_documents(query_text)

    # Convertir a DataFrame
    df_results = pd.DataFrame([
        {
            'title': doc.metadata['title'],
            'description': doc.metadata['description'],
            'link': doc.metadata['link'],
            'date_published': doc.metadata['published']
        }
        for doc in docs
    ])

    print(f"✅ {len(df_results)} resultados encontrados\n")
    return df_results

print("✅ Función de búsqueda LangChain definida")

✅ Función de búsqueda LangChain definida


### 🧪 Testing del Pipeline de LangChain

In [32]:
# Test con LangChain
query_test = "Noticias sobre deportes y fútbol"
results_langchain = langchain_search(query_test)

print("="*100)
print(results_langchain[['title', 'date_published']].to_string(index=False))
print("="*100)

# Guardar resultados en Drive
results_langchain.to_csv(f"{BASE_PATH}/output/langchain_results.csv", index=False, encoding='utf-8')
print(f"\n💾 Resultados guardados en: {BASE_PATH}/output/langchain_results.csv")


🔍 [LangChain] Buscando: 'Noticias sobre deportes y fútbol'

✅ 10 resultados encontrados

                                                                                                                              title                  date_published
                Golpe al crimen: caen ‘Los Furiosos del Sur’, banda criminal dedicada a la fabricación y tráfico de billetes falsos Fri, 14 Nov 2025 11:45:13 -0500
                                          Fans de Shakira duermen en los exteriores del Estadio Nacional antes del primer concierto Fri, 14 Nov 2025 12:16:54 -0500
                                            ¿Jorge Fossati se va de Universitario? Lo que se sabe sobre la situación del entrenador Fri, 14 Nov 2025 11:55:38 -0500
                                Fans de Shakira pasan la noche afuera del Estadio Nacional a la espera del primer concierto en Lima Fri, 14 Nov 2025 13:32:21 -0500
                              ¡Golpe azteca en Doha! México eliminó a Argentina por penales

/tmp/ipython-input-3496313405.py:15: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query_text)
